In [1]:
import numpy as np
import pandas as pd

import os
from pathlib import Path

from datasets import load_dataset

from sentence_transformers import SentenceTransformer

import torch

import random

import re

/home/amir/miniconda3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2").to(device).eval()

/home/amir/miniconda3/envs/nlp/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Model

In [4]:
dataset = load_dataset("wikipedia", language="fa", date="20240501")

/home/amir/miniconda3/envs/nlp/lib/python3.12/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


### Dataset

In [22]:
trainset = dataset['train'].shuffle(seed=42)

In [92]:
def clean_text(text):
    # Remove unicode sequences like \u200c
    text = re.sub(r'\\u[\dA-Fa-f]{4}', '', text)
    
    # Remove HTML-like tags and special sequences
    text = re.sub(r'\[\[.*?\]\]', '', text)
    text = re.sub(r'\{\{.*?\}\}', '', text)
    text = re.sub(r'\(\(.*?\)\)', '', text)
    
    # Remove multiple newlines and trim extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [145]:
sample_dataset = trainset.select(range(100000))

### Encoding

In [146]:
def encode_texts(texts):
    with torch.no_grad():
        embeddings = model.encode(texts, convert_to_tensor=True)
    return embeddings

In [147]:
texts = [clean_text(article['text']) for article in sample_dataset]
embeddings = encode_texts(texts)

- 100,000 : 6m 12s

### Query

In [206]:
idx = random.randint(0, len(sample_dataset))
query_text = clean_text(sample_dataset['text'][idx])
query = encode_texts([query_text])

### Search

In [207]:
similarities = model.similarity(query, embeddings)
values, indices = similarities.sort(descending=True)
similarities

tensor([[ 0.1504, -0.1258, -0.0698,  ...,  0.1437,  0.1223,  0.2867]],
       device='cuda:0')

### Results

In [208]:
print('Values: ', values)
print('Indices: ', indices)

Values:  tensor([[ 1.0000,  0.8753,  0.8620,  ..., -0.2429, -0.2560, -0.2595]],
       device='cuda:0')
Indices:  tensor([[ 8532, 78735, 58694,  ..., 89119, 87678, 89265]], device='cuda:0')


In [209]:
query_text == sample_dataset['text'][indices[0][0]]

False

In [210]:
neighbors = [sample_dataset['text'][i] for i in indices[0][:10]]
titles = [sample_dataset['title'][i] for i in indices[0][:10]]

df = pd.DataFrame({'Title': titles, 'Text': neighbors, 'Similarity': values[0][:10].tolist()})
df

,Title,Text,Similarity
0,فرودگاه سانوهه تانگشان,فرودگاه سانوهه تانگشان یک فرودگاه نظامی و همگ...,1.000000
1,فرودگاه تیانشویی میجیشان,فرودگاه تیانشویی میجیشان یک فرودگاه نظامی و ه...,0.875331
2,فرودگاه لیجیانگ سانی,فرودگاه لیجیانگ سانی یک فرودگاه همگانی با کد ...,0.862040
3,فرودگاه تنگچونگ توفنگ,فرودگاه تنگچونگ توفنگ یک فرودگاه همگانی با کد...,0.856722
4,فرودگاه تونگرن فنگ‌هوانگ,فرودگاه تونگرن فنگ‌هوانگ یک فرودگاه همگانی با...,0.852714
5,پایگاه هوایی هسینچو,پایگاه هوایی هسینچو یک فرودگاه نظامی است که ی...,0.850969
6,فرودگاه پینگتونگ,فرودگاه پینگتونگ یک فرودگاه همگانی با کد یاتا...,0.849332
7,فرودگاه کینهوانگدو بیدایهه,فرودگاه کینهوانگدو بیدایهه یک فرودگاه همگانی ...,0.845910
8,فرودگاه بول آلاشانکوو,فرودگاه بول آلاشانکوو یک فرودگاه همگانی با کد...,0.839918
9,فرودگاه هنگیواگ نانیو,فرودگاه هنگیواگ نانیو یک فرودگاه همگانی است ....,0.838365


### Save Cool Ones

In [165]:
df1 = df
df1

,Title,Text,Similarity
0,اچ‌ام‌اس کوراگوکس (۱۸۰۰),اچ‌ام‌اس کوراگوکس (۱۸۰۰) یک کشتی بود که طول آ...,1.000000
1,اچ‌ام‌اس ترافالگار (۱۸۲۰),اچ‌ام‌اس ترافالگار (۱۸۲۰) یک کشتی بود که طول ...,0.901638
2,اچ‌ام‌اس پلانتاگنت (۱۸۱),اچ‌ام‌اس پلانتاگنت (۱۸۱) یک کشتی بود که طول آ...,0.899385
3,اچ‌ام‌اس استریچ (۱۹۰۰),اچ‌ام‌اس استریچ (۱۹۰۰) یک کشتی بود که طول آن ...,0.895153
4,اچ‌ام‌اس مالابار (۱۸۱۸),اچ‌ام‌اس مالابار (۱۸۱۸) یک کشتی بود که طول آن...,0.895036
5,اچ‌ام‌اس لدا (۱۸۰۰),اچ‌ام‌اس لدا (۱۸۰۰) یک کشتی بود که طول آن بو...,0.890634
6,اچ‌ام‌اس دومینیکا (۱۸۰۷),اچ‌ام‌اس دومینیکا (۱۸۰۷) یک کشتی بود. این کشت...,0.885752
7,اچ‌ام‌ای‌اس کوروا,اچ‌ام‌ای‌اس کوروا یک کشتی است که طول آن می‌ب...,0.884685
8,اچ‌ام‌اس نیل (۱۸۳۹),اچ‌ام‌اس نیل (۱۸۳۹) یک کشتی بود که طول آن بو...,0.883546
9,اچ‌ام‌اس نلسون (۱۸۱۴),اچ‌ام‌اس نلسون (۱۸۱۴) یک کشتی بود که طول آن ...,0.877340


In [171]:
with pd.option_context('display.max_colwidth', None):
  display(df1)

,Title,Text,Similarity
0,اچ‌ام‌اس کوراگوکس (۱۸۰۰),اچ‌ام‌اس کوراگوکس (۱۸۰۰) یک کشتی بود که طول آن بود. این کشتی در سال ۱۸۰۰ ساخته شد.\n\nمنابع \n\n \n\nکشتی‌های ۱۸۰۰ (میلادی)\nکشتی‌های ساخت دتفورد\nکشتی‌های نیروی دریایی پادشاهی بریتانیا با کاربری رزم خطی,1.000000
1,اچ‌ام‌اس ترافالگار (۱۸۲۰),اچ‌ام‌اس ترافالگار (۱۸۲۰) یک کشتی بود که طول آن بود. این کشتی در سال ۱۸۲۰ ساخته شد.\n\nمنابع \n\n \n\nکشتی‌های ۱۸۲۰ (میلادی)\nکشتی‌های ساخت چاتهام، کنت\nکشتی‌های نیروی دریایی پادشاهی بریتانیا با کاربری رزم خطی,0.901638
2,اچ‌ام‌اس پلانتاگنت (۱۸۱),اچ‌ام‌اس پلانتاگنت (۱۸۱) یک کشتی بود که طول آن بود. این کشتی در سال ۱۸۰۱ ساخته شد.\n\nمنابع \n\n \n\nکشتی‌های ۱۸۰۱ (میلادی)\nکشتی‌های پادشاهی متحده در جنگ ۱۸۱۲\nکشتی‌های نیروی دریایی پادشاهی بریتانیا با کاربری رزم خطی,0.899385
3,اچ‌ام‌اس استریچ (۱۹۰۰),اچ‌ام‌اس استریچ (۱۹۰۰) یک کشتی بود که طول آن Length overall بود. این کشتی در سال ۱۹۰۰ (میلادی)|۱۹۰۰]] ساخته شد.\n\nمنابع \n\n \n\nکشتی‌های ۱۹۰۰ (میلادی)\nکشتی‌های ساخت گووان\nناوشکن‌های بریتانیا در جنگ جهانی اول\nناوشکن‌های کلاس سی (۱۹۱۳),0.895153
4,اچ‌ام‌اس مالابار (۱۸۱۸),اچ‌ام‌اس مالابار (۱۸۱۸) یک کشتی بود که طول آن بود. این کشتی در سال ۱۸۱۸ ساخته شد.\n\nمنابع \n\n \n\nکشتی‌های ۱۸۱۸ (میلادی)\nکشتی‌های ساخت هند بریتانیا\nکشتی‌های نیروی دریایی پادشاهی بریتانیا با کاربری رزم خطی,0.895036
5,اچ‌ام‌اس لدا (۱۸۰۰),اچ‌ام‌اس لدا (۱۸۰۰) یک کشتی بود که طول آن بود. این کشتی در سال ۱۸۰۰ ساخته شد.\n\nمنابع \n\n \n\nسوانح دریایی در ۱۸۰۸ (میلادی)\nغرق‌شدگی‌های ولز\nکشتی‌های ۱۸۰۰ (میلادی)\nکشتی‌های ساخت انگلستان\nناوچه‌های کلاس لدا\nناوچه‌های نیروی دریایی پادشاهی بریتانیا,0.890634
6,اچ‌ام‌اس دومینیکا (۱۸۰۷),اچ‌ام‌اس دومینیکا (۱۸۰۷) یک کشتی بود. این کشتی در سال ۱۸۰۷ ساخته شد.\n\nمنابع \n\n \n\nسوانح دریایی در ۱۸۰۹ (میلادی)\nکشتی‌های ۱۸۰۷ (میلادی)\nکشتی‌های دو دکله نیروی دریایی پادشاهی بریتانیا\nکشتی‌های ضبط شده,0.885752
7,اچ‌ام‌ای‌اس کوروا,اچ‌ام‌ای‌اس کوروا یک کشتی است که طول آن می‌باشد. این کشتی در سال ۱۹۱۹ ساخته شد.\n\nمنابع \n\n \n\nکشتی‌های ۱۹۱۹ (میلادی)\nکشتی‌های مین‌روب نیروی دریایی سلطنتی استرالیا,0.884685
8,اچ‌ام‌اس نیل (۱۸۳۹),اچ‌ام‌اس نیل (۱۸۳۹) یک کشتی بود که طول آن بود. این کشتی در سال ۱۸۳۹ ساخته شد.\n\nمنابع \n\nکشتی‌های ۱۸۳۹ (میلادی)\nکشتی‌های ساخت پلیموث\nکشتی‌های نیروی دریایی پادشاهی بریتانیا با کاربری رزم خطی,0.883546
9,اچ‌ام‌اس نلسون (۱۸۱۴),اچ‌ام‌اس نلسون (۱۸۱۴) یک کشتی بود که طول آن بود. این کشتی در سال ۱۸۱۴ ساخته شد.\n\nمنابع\n\nپیوند به بیرون \n\nکشتی‌های ۱۸۱۴ (میلادی)\nکشتی‌های نیروی دریایی پادشاهی بریتانیا با کاربری رزم خطی\nسوانح دریایی در ۱۹۰۸ (میلادی),0.877340


In [172]:
df2 = df
df2

,Title,Text,Similarity
0,اینوسور,اینوسور (نام علمی: Inosaurus) نام یک سرده از ر...,1.000000
1,بروهیسور,بروهیسور نام یک سرده از راسته خزنده‌کفلان است....,0.863796
2,کیلسور,کیلسور نام یک سرده از سخت‌دنبان است.\n\nمنابع...,0.861113
3,فرگاناسور,فرگاناسور (نام علمی: Ferganasaurus verzilini)...,0.859491
4,ایگناووسور,ایگناووسور (نام علمی: Ignavusaurus) نام یک سرد...,0.845444
5,زبی,زبی نام یک گونه از فروراسته خزنده‌پایان است.\...,0.844518
6,آتسینگانوسور,آتسینگانوسور (نام علمی: Atsinganosaurus) نام ی...,0.843637
7,آتلاسور,آتلاسور (نام علمی: Atlasaurus imelakei) نام یک...,0.830198
8,تک‌پنجه,تک‌پنجه نام یک گونه از تیره آلوارزخزندگان است....,0.821092
9,نواسور,نواسور (نام علمی: Noasaurus) نام یک گونه از کل...,0.818271


In [173]:
with pd.option_context('display.max_colwidth', None):
  display(df2)

,Title,Text,Similarity
0,اینوسور,اینوسور (نام علمی: Inosaurus) نام یک سرده از راسته خزنده‌کفلان است.\n\nمنابع \n\n \n\nدایناسورهای آفریقا\nدایناسورهای کرتاسه\nددپایان\nسنگواره‌های توصیف‌شده در ۱۹۶۰ (میلادی)\nسنگواره‌های نیجر\nددپایان پیشاتاریخ,1.000000
1,بروهیسور,بروهیسور نام یک سرده از راسته خزنده‌کفلان است.\n\nمنابع \n\n \n\nدایناسورهای آسیا\nدایناسورهای ژوراسیک\nدایناسورهای ژوراسیک پسین آسیا\nسنگواره‌های توصیف‌شده در ۲۰۰۳ (میلادی)\nسوسمارپایان\nسنگواره‌های پاکستان,0.863796
2,کیلسور,کیلسور نام یک سرده از سخت‌دنبان است.\n\nمنابع\n\nپیوند به بیرون \n\nبازبینی گمر زیست‌شناسی\nدایناسورهای آمریکای جنوبی\nدایناسورهای ژوراسیک\nدایناسورهای ژوراسیک پسین\nددپایان\nژوراسیک شیلی\nسنگواره‌های توصیف‌شده در ۲۰۱۵ (میلادی)\nسنگواره‌های شیلی\nددپایان پیشاتاریخ,0.861113
3,فرگاناسور,فرگاناسور (نام علمی: Ferganasaurus verzilini) نام یک سرده از زیرراسته خزنده‌پاریختان است.\n\nمنابع \n\n \n\nدایناسورهای آسیا\nدایناسورهای ژوراسیک\nدایناسورهای ژوراسیک میانه آسیا\nسنگواره‌های توصیف‌شده در ۲۰۰۳ (میلادی)\nسوسمارپایان\nسنگواره‌های قرقیزستان,0.859491
4,ایگناووسور,ایگناووسور (نام علمی: Ignavusaurus) نام یک سرده از زیرراسته سوسمارپاشکلان است.\n\nمنابع \n\n \n\nدایناسورهای آفریقا\nدایناسورهای ژوراسیک\nدیرین‌شناسی در لسوتو\nسنگواره‌های توصیف‌شده در ۲۰۱۰ (میلادی)\nسوسمارپاسانان,0.845444
5,زبی,زبی نام یک گونه از فروراسته خزنده‌پایان است.\n\nمنابع \n\n \n\nدایناسورهای اروپا\nدایناسورهای ژوراسیک\nدایناسورهای ژوراسیک پسین اروپا\nدیرین‌شناسی در پرتغال\nسنگواره‌های توصیف‌شده در ۲۰۱۴ (میلادی)\nسوسمارپایان\nسنگواره‌های پرتغال,0.844518
6,آتسینگانوسور,آتسینگانوسور (نام علمی: Atsinganosaurus) نام یک سرده از زیرراسته سوسمارپاشکلان است.\n\nمنابع \n\n \n\nتایتانوسورها\nدایناسورهای اروپا\nدایناسورهای کرتاسه\nدایناسورهای کرتاسه پسین اروپا\nدیرین‌شناسی در فرانسه\nزیست در دور ماستریختین\nسنگواره‌های توصیف‌شده در ۲۰۱۰ (میلادی)\nسنگواره‌های فرانسه,0.843637
7,آتلاسور,آتلاسور (نام علمی: Atlasaurus imelakei) نام یک گونه از راسته خزنده‌کفلان است.\n\nمنابع \n\n \n\nبراکیوسور\nدایناسورهای آفریقا\nدایناسورهای ژوراسیک\nدایناسورهای ژوراسیک میانه آفریقا\nسنگواره‌های توصیف‌شده در ۱۹۹۹ (میلادی)\nسنگواره‌های مراکش,0.830198
8,تک‌پنجه,تک‌پنجه نام یک گونه از تیره آلوارزخزندگان است.\n\nمنابع \n\n \n\nدایناسورهای آسیا\nدایناسورهای کرتاسه\nدایناسورهای کرتاسه پسین آسیا\nسنگواره‌های توصیف‌شده در ۱۹۹۳ (میلادی),0.821092
9,نواسور,نواسور (نام علمی: Noasaurus) نام یک گونه از کلاد ددپایان است.\n\nمنابع \n\n \n\nدایناسورهای آمریکای جنوبی\nدایناسورهای کرتاسه\nدایناسورهای کرتاسه پسین آمریکای جنوبی\nسنگواره‌های توصیف‌شده در ۱۹۸۰ (میلادی)\nشاخ‌سوسماران,0.818271


In [179]:
df3 = df
df3

,Title,Text,Similarity
0,شرفه (استان عنابه),شرفه یک شهرداری در الجزایر است که در استان عن...,1.000000
1,تملاحت,تملاحت یک شهرداری در الجزایر است که در استان ...,0.945868
2,سریانه,سریانه یک شهرداری در الجزایر است که در استان ...,0.929130
3,سبت (تیارت),سبت یک شهرداری در الجزایر است که در استان تیا...,0.923265
4,محره,محره یک شهرداری در الجزایر است که در استان ال...,0.912855
5,ولجه (استان غلیزان),ولجه یک شهرداری در الجزایر است که در استان غل...,0.907282
6,رقاصه,رقاصه یک شهرداری در الجزایر است که در استان ا...,0.904615
7,شط (استان طارف),شط یک شهرداری در الجزایر است که در استان الطا...,0.902320
8,جندل (سکیکده),جندل یک شهرداری در الجزایر است که در استان سک...,0.898597
9,ملعب,ملعب یک شهرداری در الجزایر است که در استان تس...,0.894971


In [180]:
with pd.option_context('display.max_colwidth', None):
  display(df3)

,Title,Text,Similarity
0,شرفه (استان عنابه),شرفه یک شهرداری در الجزایر است که در استان عنابه واقع شده‌است.\n\nجستارهای وابسته \n فهرست شهرهای الجزایر\n\nمنابع\n\nپیوند به بیرون \n\nشهرداری‌های استان عنابه,1.000000
1,تملاحت,تملاحت یک شهرداری در الجزایر است که در استان تسمسیلت واقع شده‌است.\n\nجستارهای وابسته \n فهرست شهرهای الجزایر\n\nمنابع\n\nپیوند به بیرون \n\nشهرداری‌های استان\nمناطق مسکونی در استان تسمسیلت,0.945868
2,سریانه,سریانه یک شهرداری در الجزایر است که در استان باتنه واقع شده‌است.\n\nجستارهای وابسته \n فهرست شهرهای الجزایر\n\nمنابع\n\nپیوند به بیرون \n\nشهرداری‌های استان\nمناطق مسکونی در استان باتنه,0.929130
3,سبت (تیارت),سبت یک شهرداری در الجزایر است که در استان تیارت واقع شده‌است.\n\nجستارهای وابسته \n فهرست شهرهای الجزایر\n\nمنابع\n\nپیوند به بیرون \n\nشهرداری‌های استان\nمناطق مسکونی در استان تیارت\nشهرهای الجزایر,0.923265
4,محره,محره یک شهرداری در الجزایر است که در استان البیض واقع شده‌است.\n\nجستارهای وابسته \nفهرست شهرهای الجزایر\n\nمنابع\n\nپیوند به بیرون \n\nمناطق مسکونی در استان البیض\nشهرداری‌های استان\nشهرداری‌های استان البیض,0.912855
5,ولجه (استان غلیزان),ولجه یک شهرداری در الجزایر است که در استان غلیزان واقع شده‌است.\n\nجستارهای وابسته \n فهرست شهرهای الجزایر\n\nمنابع\n\nپیوند به بیرون \n\nشهرداری‌های استان\nمناطق مسکونی در استان غلیزان,0.907282
6,رقاصه,رقاصه یک شهرداری در الجزایر است که در استان البیض واقع شده‌است.\n\nجستارهای وابسته \n فهرست شهرهای الجزایر\n\nمنابع\n\nپیوند به بیرون \n\nمناطق مسکونی در استان البیض\nشهرداری‌های استان\nشهرهای الجزایر\nشهرداری‌های استان البیض,0.904615
7,شط (استان طارف),شط یک شهرداری در الجزایر است که در استان الطارف واقع شده‌است.\n\nجستارهای وابسته \n فهرست شهرهای الجزایر\n\nمنابع\n\nپیوند به بیرون \n\nشهرداری‌های استان\nمناطق مسکونی در استان الطارف,0.902320
8,جندل (سکیکده),جندل یک شهرداری در الجزایر است که در استان سکیکده واقع شده‌است.\n\nجستارهای وابسته \n فهرست شهرهای الجزایر\n\nمنابع\n\nپیوند به بیرون \n\nمناطق مسکونی در استان سکیکده\nشهرداری‌های استان,0.898597
9,ملعب,ملعب یک شهرداری در الجزایر است که در استان تسمسیلت واقع شده‌است.\n\nجستارهای وابسته \nفهرست شهرهای الجزایر\n\nمنابع\n\nپیوند به بیرون \n\nمناطق مسکونی در استان تسمسیلت\nشهرداری‌های استان\nشهرهای الجزایر,0.894971
